# Introduction to Notebook 
This notebook shows a handson session of the application of Topic Modelling using Gensim, a popular python library. The notebook will explain how to apply Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation (LDA)​ algorithm and how to measure the accuracy of the topic modelling model using the coherence score. 
Our work is captured here:
https://www.academia.edu/19204202/Concept_driven_Classification_and_Extraction_of_technical_help_documents 

Clustering using Glove
https://www.linkedin.com/pulse/unsupervised-learning-glove-word-embeddings-search-queries-bhandari/ 



# Dataset
We will use Query Dataset

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#dataset_df = pd.DataFrame({'News':dataset.data, 'Label' : dataset.target})
dataset_df = pd.read_csv("/content/drive/MyDrive/Tutorials/VJIET/Day2/QueryLabels.tsv",dtype=str,encoding='ISO-8859-1',sep='\t')

In [ ]:
dataset_df

,Query,Label,Label_names
0,internet explorer,2,ie
1,windowsmedia,1,Windows
2,microsoft,8,Others
3,outlook.com,3,Office
4,Preview window,1,Windows
...,...,...,...
494,www.outlook.com email,3,Office
495,secure boot isn't configured correctly 8.1,1,Windows
496,smartglass,5,xbox
497,how to print screen,1,Windows


In [ ]:
dataset_df

,Query,Label,Label_names
0,internet explorer,2,ie
1,windowsmedia,1,Windows
2,microsoft,8,Others
3,outlook.com,3,Office
4,Preview window,1,Windows
...,...,...,...
494,www.outlook.com email,3,Office
495,secure boot isn't configured correctly 8.1,1,Windows
496,smartglass,5,xbox
497,how to print screen,1,Windows


# Dataset preprocessing

In [ ]:
%%capture
!pip install -U gensim

In [ ]:
from gensim.utils import tokenize
from gensim.parsing.preprocessing import preprocess_string,strip_tags,strip_punctuation,strip_numeric,remove_stopwords,strip_short
from gensim.corpora.dictionary import Dictionary
from gensim import models

In [ ]:
help(preprocess_string)

Help on function preprocess_string in module gensim.parsing.preprocessing:

preprocess_string(s, filters=[<function <lambda> at 0x7f006b2da290>, <function strip_tags at 0x7f006ece3680>, <function strip_punctuation at 0x7f006ece3290>, <function strip_multiple_whitespaces at 0x7f006eceb560>, <function strip_numeric at 0x7f006ece3ef0>, <function remove_stopwords at 0x7f006ecf7ef0>, <function strip_short at 0x7f006ece3830>, <function stem_text at 0x7f006b2da0e0>])
    Apply list of chosen filters to `s`.
    
    Default list of filters:
    
    * :func:`~gensim.parsing.preprocessing.strip_tags`,
    * :func:`~gensim.parsing.preprocessing.strip_punctuation`,
    * :func:`~gensim.parsing.preprocessing.strip_multiple_whitespaces`,
    * :func:`~gensim.parsing.preprocessing.strip_numeric`,
    * :func:`~gensim.parsing.preprocessing.remove_stopwords`,
    * :func:`~gensim.parsing.preprocessing.strip_short`,
    * :func:`~gensim.parsing.preprocessing.stem_text`.
    
    Parameters
    -------

In [ ]:
dataset_df['Clean_Queries'] = dataset_df['Query'].apply(preprocess_string)

In [ ]:
dataset_df

,Query,Label,Label_names,Clean_Queries
0,internet explorer,2,ie,"[internet, explor]"
1,windowsmedia,1,Windows,[windowsmedia]
2,microsoft,8,Others,[microsoft]
3,outlook.com,3,Office,"[outlook, com]"
4,Preview window,1,Windows,"[preview, window]"
...,...,...,...,...
494,www.outlook.com email,3,Office,"[www, outlook, com, email]"
495,secure boot isn't configured correctly 8.1,1,Windows,"[secur, boot, isn, configur, correctli]"
496,smartglass,5,xbox,[smartglass]
497,how to print screen,1,Windows,"[print, screen]"


In [ ]:
filters=[lambda x: x.lower(),strip_tags,strip_punctuation,remove_stopwords,strip_short]
dataset_df['Clean_Queries1'] = dataset_df['Query'].apply(lambda x: preprocess_string(x,filters))

In [ ]:
dataset_df

,Query,Label,Label_names,Clean_Queries,Clean_Queries1
0,internet explorer,2,ie,"[internet, explor]","[internet, explorer]"
1,windowsmedia,1,Windows,[windowsmedia],[windowsmedia]
2,microsoft,8,Others,[microsoft],[microsoft]
3,outlook.com,3,Office,"[outlook, com]","[outlook, com]"
4,Preview window,1,Windows,"[preview, window]","[preview, window]"
...,...,...,...,...,...
494,www.outlook.com email,3,Office,"[www, outlook, com, email]","[www, outlook, com, email]"
495,secure boot isn't configured correctly 8.1,1,Windows,"[secur, boot, isn, configur, correctli]","[secure, boot, isn, configured, correctly]"
496,smartglass,5,xbox,[smartglass],[smartglass]
497,how to print screen,1,Windows,"[print, screen]","[print, screen]"


In [ ]:
dataset_dictionary = Dictionary(dataset_df['Clean_Queries1'])

In [ ]:
len(dataset_dictionary)

356

In [ ]:
print(dataset_dictionary.token2id)

{'explorer': 0, 'internet': 1, 'windowsmedia': 2, 'microsoft': 3, 'com': 4, 'outlook': 5, 'preview': 6, 'window': 7, 'homepage': 8, 'msn': 9, 'hotm': 10, 'skype': 11, 'account': 12, 'google': 13, 'toolbar': 14, 'history': 15, 'manage': 16, 'flash': 17, 'object': 18, 'ocx': 19, 'shockwave': 20, 'windows': 21, 'java': 22, 'office': 23, 'setup': 24, 'www': 25, 'media': 26, 'player': 27, 'update': 28, 'settings': 29, 'download': 30, 'free': 31, 'app': 32, 'store': 33, 'sharepoint': 34, 'marinenet': 35, 'mil': 36, 'usmc': 37, 'essentials': 38, 'security': 39, 'login': 40, 'downloads': 41, '2147006': 42, 'article': 43, 'citeworld': 44, 'consumerization': 45, 'fall': 46, 'html': 47, 'rehabilitation': 48, 'rise': 49, 'office365': 50, 'defender': 51, 'backgrounds': 52, 'desktop': 53, 'menu': 54, 'w2express': 55, 'support': 56, 'firefox': 57, 'mozilla': 58, 'live': 59, 'password': 60, 'reset': 61, 'background': 62, 'xbox': 63, 'maker': 64, 'movie': 65, 'help': 66, 'word': 67, 'mail': 68, '360': 

In [ ]:
dataset_corpus_bow = [dataset_dictionary.doc2bow(text) for text in dataset_df['Clean_Queries1']] #create a dataset corpus with bag of word vectorization

In [ ]:
len(dataset_corpus_bow)

499

In [ ]:
#Preview window
print(dataset_corpus_bow[4])

[(6, 1), (7, 1)]


In [ ]:
tfidf = models.TfidfModel(dataset_corpus_bow)
dataset_corpus_tfidf = tfidf[dataset_corpus_bow]

In [ ]:
len(dataset_corpus_tfidf)

499

In [ ]:
print(dataset_corpus_tfidf[4])

[(6, 0.6641717269945677), (7, 0.7475800405716124)]


# Topic Modelling with Latent Dirichlet Allocation(LDA)
LDA is one of the popular techniques for topic modelling.
Each document consists of various words and each topic can be associated with some words. The aim behind the LDA to find topics that the document belongs to, on the basis of words contains in it. It assumes that documents with similar topics will use a similar group of words. This enables the documents to map the probability distribution over latent topics and topics are probability distribution.

https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2 

In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
lda_bow = LdaModel(dataset_corpus_bow,num_topics=20,id2word=dataset_dictionary,random_state=0)

In [ ]:
lda_topics_bow = lda_bow.print_topics(num_words=8)
for topic in lda_topics_bow:
  print(topic)

(0, '0.073*"windows" + 0.073*"explorer" + 0.061*"internet" + 0.037*"microsoft" + 0.032*"com" + 0.032*"xbox" + 0.025*"rewards" + 0.025*"extension"')
(1, '0.085*"microsoft" + 0.035*"flash" + 0.035*"desktop" + 0.035*"ultipro" + 0.035*"com" + 0.035*"history" + 0.018*"object" + 0.018*"shockwave"')
(2, '0.074*"download" + 0.055*"office" + 0.037*"panel" + 0.037*"control" + 0.037*"email" + 0.019*"msvcr" + 0.019*"skype" + 0.019*"add"')
(3, '0.104*"sign" + 0.035*"file" + 0.035*"com" + 0.035*"extension" + 0.035*"outlook" + 0.035*"msn" + 0.018*"web" + 0.018*"home"')
(4, '0.130*"microsoft" + 0.095*"download" + 0.024*"net" + 0.024*"ssv" + 0.024*"web" + 0.024*"office" + 0.024*"support" + 0.024*"free"')
(5, '0.106*"update" + 0.061*"explorer" + 0.031*"homepage" + 0.031*"server" + 0.031*"cmd" + 0.031*"mail" + 0.031*"outlook" + 0.031*"bing"')
(6, '0.176*"explorer" + 0.176*"internet" + 0.047*"download" + 0.028*"xbox" + 0.019*"adblock" + 0.019*"update" + 0.019*"google" + 0.010*"bit"')
(7, '0.095*"windows" 

In [ ]:
lda_tfidf = LdaModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [ ]:
lda_topics_tfidf = lda_tfidf.print_topics(num_words=8)
for topic in lda_topics_tfidf:
  print(topic)

(0, '0.128*"update" + 0.036*"windows" + 0.026*"explorer" + 0.023*"skype" + 0.023*"live" + 0.023*"internetexplorer" + 0.023*"drive" + 0.022*"openoffice"')
(1, '0.038*"movie" + 0.038*"windowsonecare" + 0.038*"favorites" + 0.030*"backgrounds" + 0.029*"csrss" + 0.028*"access" + 0.027*"iso" + 0.027*"usb"')
(2, '0.040*"internet" + 0.034*"windows" + 0.032*"outlook" + 0.026*"explorer" + 0.025*"desktop" + 0.021*"net" + 0.020*"office" + 0.018*"open"')
(3, '0.034*"com" + 0.034*"skype" + 0.032*"windows" + 0.030*"www" + 0.023*"mse" + 0.023*"iexplorer" + 0.023*"theme" + 0.023*"keyboard"')
(4, '0.070*"office" + 0.033*"download" + 0.031*"wallpaper" + 0.030*"microsoft" + 0.028*"sign" + 0.027*"explorer" + 0.026*"screensavers" + 0.026*"windows"')
(5, '0.054*"com" + 0.035*"internet" + 0.034*"explorer" + 0.029*"screen" + 0.029*"java" + 0.027*"office" + 0.027*"outlook" + 0.026*"account"')
(6, '0.044*"app" + 0.042*"enable" + 0.042*"store" + 0.038*"cookies" + 0.024*"windows" + 0.020*"encarta" + 0.020*"shut" +

# Topic Modelling with Latent Semantic Analysis/Indexing(LSA/LSI)

Latent Semantic Analysis is a natural language processing method that analyzes relationships between a set of documents and the terms contained within. It uses singular value decomposition, a mathematical technique, to scan unstructured data to find hidden relationships between terms and concepts.

In [ ]:
from gensim.models.lsimodel import LsiModel

In [ ]:
lsi_bow = LsiModel(corpus=dataset_corpus_bow,id2word=dataset_dictionary,num_topics=20)

In [ ]:
lsi_topics_bow = lsi_bow.print_topics(num_words=8)
for topic in lsi_topics_bow:
  print(topic)

(0, '-0.905*"windows" + -0.276*"download" + -0.146*"explorer" + -0.126*"internet" + -0.096*"microsoft" + -0.080*"free" + -0.068*"live" + -0.066*"update"')
(1, '-0.628*"microsoft" + -0.548*"com" + -0.259*"www" + -0.235*"office" + 0.201*"windows" + -0.169*"outlook" + -0.151*"explorer" + -0.145*"internet"')
(2, '-0.674*"explorer" + -0.640*"internet" + 0.221*"microsoft" + 0.172*"windows" + 0.089*"com" + -0.088*"update" + 0.062*"office" + 0.056*"www"')
(3, '0.701*"file" + 0.701*"extension" + 0.028*"dll" + 0.028*"exe" + 0.027*"qfx" + 0.027*"mui" + 0.027*"ofx" + 0.027*"deskthemepack"')
(4, '0.662*"microsoft" + -0.555*"com" + -0.277*"outlook" + -0.272*"www" + -0.117*"email" + -0.111*"xbox" + 0.089*"login" + -0.086*"live"')
(5, '-0.893*"download" + 0.243*"windows" + -0.206*"free" + -0.146*"skype" + 0.136*"microsoft" + 0.089*"internet" + 0.078*"explorer" + 0.072*"update"')
(6, '0.708*"outlook" + 0.314*"office" + -0.302*"com" + 0.255*"sign" + -0.224*"xbox" + 0.203*"email" + 0.168*"log" + 0.158*"w

In [ ]:
lsi_tfidf = LsiModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [ ]:
lsi_topics_tfidf = lsi_tfidf.print_topics(num_words=8)
for topic in lsi_topics_tfidf:
  print(topic)

(0, '0.535*"explorer" + 0.506*"windows" + 0.446*"internet" + 0.386*"download" + 0.176*"update" + 0.102*"microsoft" + 0.088*"live" + 0.078*"com"')
(1, '-0.456*"microsoft" + -0.375*"com" + -0.323*"outlook" + 0.321*"explorer" + -0.300*"office" + 0.284*"internet" + -0.226*"sign" + -0.204*"www"')
(2, '0.687*"windows" + -0.375*"explorer" + -0.349*"internet" + -0.215*"outlook" + -0.209*"com" + -0.188*"microsoft" + -0.163*"office" + 0.132*"download"')
(3, '-0.576*"microsoft" + 0.435*"outlook" + -0.368*"download" + 0.249*"com" + 0.211*"xbox" + 0.192*"windows" + -0.186*"login" + 0.185*"live"')
(4, '0.717*"download" + -0.420*"microsoft" + -0.249*"windows" + 0.230*"skype" + 0.177*"outlook" + -0.146*"update" + -0.140*"essentials" + -0.113*"live"')
(5, '0.653*"xbox" + -0.418*"outlook" + 0.400*"live" + -0.219*"office" + -0.195*"sign" + 0.166*"com" + 0.149*"games" + -0.147*"windows"')
(6, '0.684*"file" + 0.684*"extension" + 0.055*"dll" + 0.055*"exe" + 0.051*"kmz" + 0.051*"diagcab" + 0.051*"deskthemepa

# Topic Modelling Visualization with pyLDAvis

In [ ]:
%%capture
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis_bow = pyLDAvis.gensim_models.prepare(lda_bow, dataset_corpus_bow, dataset_dictionary)
vis_bow

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.133711 -0.127481       1        1  8.273020
6     -0.198942 -0.169213       2        1  7.559503
4      0.109823 -0.105334       3        1  6.666041
17     0.029855  0.001427       4        1  6.637888
7     -0.130377  0.056090       5        1  6.367868
16     0.005933 -0.036923       6        1  5.388812
8     -0.006335  0.147370       7        1  5.349535
18     0.124478  0.032480       8        1  5.076869
10    -0.050826 -0.020281       9        1  4.839063
3      0.070870  0.046029      10        1  4.628294
14    -0.078068  0.078344      11        1  4.583058
13     0.088467  0.038898      12        1  4.438388
9      0.013437  0.123157      13        1  4.229365
1      0.013588 -0.094952      14        1  4.105673
0     -0.114282 -0.046640      15        1  4.058553
5     -0.101645 -0.044880      16        1  3.760848
12    -0.058427  0.166576      17        1  3.713260
2      0.052648  0.033283      18        1  3.654862
15    -0.024524 -0.033335      19        1  3.555874
19     0.120616 -0.044618      20        1  3.113225, topic_info=        Term       Freq      Total Category  logprob  loglift
0   explorer  24.000000  24.000000  Default  30.0000  30.0000
1   internet  22.000000  22.000000  Default  29.0000  29.0000
21   windows  55.000000  55.000000  Default  28.0000  28.0000
30  download  29.000000  29.000000  Default  27.0000  27.0000
57      live  14.000000  14.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
21   windows   0.035559  55.136087  Topic20  -6.9006  -3.8769
30  download   0.035559  29.467283  Topic20  -6.9006  -3.2503
0   explorer   0.035559  24.595158  Topic20  -6.9006  -3.0696
1   internet   0.035559  22.700548  Topic20  -6.9006  -2.9894
23    office   0.035559  18.229828  Topic20  -6.9006  -2.7701

[744 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
190       1  0.313534    access
190       3  0.313534    access
190      16  0.313534    access
12        4  0.134772   account
12        8  0.134772   account
...     ...       ...       ...
61       20  0.069891      xbox
144       5  0.587741    xboxie
313       4  0.669854  xboxlive
265       9  0.694682     yahoo
157       2  0.641101      zune

[740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 7, 5, 18, 8, 17, 9, 19, 11, 4, 15, 14, 10, 2, 1, 6, 13, 3, 16, 20])

In [ ]:
vis_tfidf = pyLDAvis.gensim_models.prepare(lda_tfidf, dataset_corpus_tfidf, dataset_dictionary)
vis_tfidf

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.016603 -0.139455       1        1  7.264077
5     -0.102806 -0.066417       2        1  6.963118
2     -0.091557  0.082974       3        1  6.606295
15    -0.001961 -0.034472       4        1  6.266543
14    -0.106837  0.009711       5        1  5.983472
9      0.036398  0.051017       6        1  5.746878
4     -0.077229  0.003746       7        1  5.352122
17     0.018723  0.079401       8        1  5.282064
8     -0.085802 -0.007568       9        1  4.935600
12    -0.022768  0.088443      10        1  4.911580
0     -0.067075  0.005399      11        1  4.805707
6      0.012216  0.040380      12        1  4.784962
18     0.116457  0.122275      13        1  4.561782
10     0.057945 -0.090404      14        1  4.528999
3      0.054673 -0.050784      15        1  4.282804
13     0.083622  0.002137      16        1  3.939085
7      0.043931 -0.026257      17        1  3.840064
19    -0.008747 -0.018664      18        1  3.827149
16     0.073182 -0.025011      19        1  3.271783
1      0.051032 -0.026452      20        1  2.845917, topic_info=         Term       Freq      Total Category  logprob  loglift
28     update   8.000000   8.000000  Default  30.0000  30.0000
61       xbox   8.000000   8.000000  Default  29.0000  29.0000
30   download  13.000000  13.000000  Default  28.0000  28.0000
23     office   9.000000   9.000000  Default  27.0000  27.0000
0    explorer  12.000000  12.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
120      sign   0.036352   7.523639  Topic20  -6.3175  -1.7733
9         msn   0.036352   5.989974  Topic20  -6.3175  -1.5453
22       java   0.036352   6.262020  Topic20  -6.3175  -1.5897
25        www   0.036352   6.398639  Topic20  -6.3175  -1.6113
28     update   0.036352   8.191426  Topic20  -6.3175  -1.8583

[779 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
190      20  0.490985    access
12        2  0.213165   account
12        4  0.213165   account
12        9  0.213165   account
12       20  0.213165   account
...     ...       ...       ...
61       10  0.119434      xbox
144      16  0.749770    xboxie
313      17  0.686616  xboxlive
265      10  0.859365     yahoo
157       7  0.736709      zune

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 6, 3, 16, 15, 10, 5, 18, 9, 13, 1, 7, 19, 11, 4, 14, 8, 20, 17, 2])

# Model evaluation for Topic Modelling

Topic coherence is a quantitative method to measure the quality of topics, how similar the top words are similar to each other and how interpretable topics are to humans.Coherence is expressed as the sum of pairwise scores on the words w1, …, wn used to describe the topic . Coherence is usually an intrinsic or extrinsic measure. For the purpose of the session, two options for coherence will be implemented using the coherence model in gensim. u_mass(a measure of how often two words were seen together with a range of-14 and 14) and c_v (0 and 1)

In [ ]:
from gensim.models import CoherenceModel

In [ ]:
texts= dataset_df['Clean_Queries1']
texts = [x for x in texts if x]

In [ ]:
cm_lda_bow_cv = CoherenceModel(model=lda_bow,texts=texts,dictionary=dataset_dictionary,coherence='c_v')
cm_lda_bow_cv.get_coherence()

0.6466434028086667

In [ ]:
cm_lsi_bow_cv = CoherenceModel(model=lsi_bow, texts=texts, dictionary=dataset_dictionary, coherence='c_v')
cm_lsi_bow_cv.get_coherence()

0.6188060197529739